In [1]:
import math
import numpy as np
import torch.distributed as dist
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import pandas as pd
import test  # import test.py to get mAP after each epoch
from models.yolo import Model
from utils.datasets import create_dataloader



import pickle
import os
from shapely.geometry import Polygon
from pyproj import Transformer
import ast

pd.options.mode.chained_assignment = None 
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [2]:
def geo_coords(box, x_translation, y_translation, geo_data, center=True):
  """
  Gets coordinates in epsg:4326 format (google maps) of bounding box
  If center, return only coordinate of center of bounding box
  Params:
    Box: [xmin, ymin, xmax, ymax]
  """
  transformer = Transformer.from_crs(geo_data['crs'], 'epsg:4326')
  if center:
    center_x, center_y = (box[2] + box[0])//2 + x_translation, (box[3] + box[1])//2 + y_translation
    new_coords = np.array(geo_data['transform']).reshape((3,3)) @ np.array( [center_x, center_y, 1])

    return transformer.transform(new_coords[0], new_coords[1])

  else:
    new_box = np.array([[x_translation + box[0], y_translation + box[1], 1],
            [x_translation + box[2], y_translation + box[3], 1]])
    new_coords = np.array(geo_data['transform']).reshape((3,3)) @ new_box.T

    return transformer.transform(new_coords[0], new_coords[1])

def get_list_polygon(xs, ys):
  xs, ys = sorted(xs), sorted(ys)
  return [[ys[0], xs[0]], [ys[1], xs[0]], [ys[1], xs[1]], [ys[0], xs[1]]]


In [5]:


def load_targets(annot_path, set_name='val'):
    df = pd.read_csv(os.path.join(annot_path, f'{set_name}.csv')
                     , index_col=0).sort_values(['directory', 'file']).reset_index(drop=True)
    df['width'], df['height'] = 256, 256
    coordinate_list = []
    for file in sorted(os.listdir(os.path.join(annot_path,'labels', set_name))):
        try:
            df_res = pd.read_csv(os.path.join(annot_path, 'labels', set_name, file), header=None, sep=' ')
            num = int(file.replace('.txt', ''))
            md = df.iloc[num]
            md['transform'] = ast.literal_eval(md['transform'].replace("'", '"'))
            for i, row in df_res.iterrows():
                width, height = row[3]* md['width'], row[4] * md['height']
                center_x, center_y = row[1] * md['width'], row[2] * md['height']
                box = [center_x- width//2, center_y - height//2, center_x + width // 2, center_y + width //2]
                xs, ys = geo_coords(box, md['x_offset'], md['y_offset'], md, center=False)
                coordinate_list.append(get_list_polygon(xs, ys))
        except Exception as e:
            pass
    return coordinate_list
    
compute = True
if compute:
    directory = 'cleancoolingTowers_negsamples'
    coord_list = load_targets(f'../datasets/{directory}', )
    import pickle
    with open(f'../datasets/{directory}/val_coords.txt', 'wb') as f:
        pickle.dump(coord_list, f)

No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns

No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns

No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns

No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns

No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns

No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns

No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns to parse from file
No columns

In [6]:
df = pd.read_csv('../datasets/cleancoolingTowers_negsamples/val.csv', index_col=0).sort_values(['directory', 'file']).reset_index(drop=True)
df['width'], df['height'] = 256, 256
df.rename(columns={'x_offset': 'x_pos', 'y_offset': 'y_pos'}, inplace=True)
import tqdm
def experiment_evaluate(exp_path, df_metadata, min_thr=0.25):
    coordinate_list=[]
    for file in tqdm.tqdm(os.listdir(os.path.join(exp_path, 'autolabels'))):
        df_res = pd.read_csv(os.path.join(exp_path, 'autolabels', file), header=None, sep=' ')
        num = int(file.replace('.txt', ''))
        md = df_metadata.loc[num]
        md['transform'] = ast.literal_eval(md['transform'].replace("'", '"'))
        df = df_res.loc[df_res[1] > min_thr]
        for i, row in df.iterrows():
            width, height = row[4]* md['width'], row[5] * md['height']
            center_x, center_y = row[2] * md['width'], row[3] * md['height']
            box = [center_x- width//2, center_y - height//2, center_x + width // 2, center_y + width //2]
            xs, ys = geo_coords(box, md['x_pos'], md['y_pos'], md, center=False)
            coordinate_list.append((get_list_polygon(xs, ys),row[1]))
    with open(f'{exp_path}/coords.pkl', 'wb') as f:
        pickle.dump(coordinate_list, f)
    return coordinate_list
preds = experiment_evaluate('runs/testexp3/', df)

100%|██████████| 1636/1636 [00:20<00:00, 81.06it/s] 


In [9]:
with open('runs/testexp23//coords.pkl', 'rb') as f:
    preds = pickle.load(f)

In [12]:
with open('../datasets/cleancoolingTowers_negsamples/val_coords.txt', 'rb') as f:
    coord_list = pickle.load(f)

import geopandas
confidences = [p[1] for p in preds]
polygons_preds = [Polygon(p[0]) for p in preds]
polygon_target = [Polygon(p) for p in coord_list]
gdf_preds = geopandas.GeoDataFrame(confidences, geometry=polygons_preds)
gdf_preds = gdf_preds[gdf_preds[0] > 0.5]
gdf_targets = geopandas.GeoDataFrame(geometry=polygon_target)
preds_match = geopandas.sjoin(gdf_preds, gdf_targets, how='left', op='intersects').drop_duplicates('geometry')
target_match = geopandas.sjoin(gdf_preds, gdf_targets, how='right', op='intersects').drop_duplicates('geometry')


tp = target_match[target_match.index_left.notnull()].drop_duplicates('index_left').shape[0]
fp = preds_match[preds_match.index_right.isna()].shape[0]
fn = target_match[target_match.index_left.isna()].shape[0]
print(f'precision {tp/(tp+fp)}, recall {tp/(tp+fn)}')

precision 0.9117647058823529, recall 0.8985507246376812
